In [1]:
import torch
import torchvision
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, CrossEntropyLoss

In [2]:
# 使用Sequential来优化网络结构
class MyModule(torch.nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()

        self.model = torch.nn.Sequential(
            Conv2d(3, 32, 5, padding=2),  # Input: 3@32×32, Output: 32@32×32
            MaxPool2d(2),  # Input: 32@32×32, Output: 32@16×16
            Conv2d(32, 32, 5, padding=2),  # Input: 32@16×16, Output: 32@16×16
            MaxPool2d(2),  # Input: 32@16×16, Output: 32@8×8
            Conv2d(32, 64, 5, padding=2),  # Input: 32@8×8, Output: 64@8×8
            MaxPool2d(2),  # Input: 64@8×8, Output: 64@4×4
            Flatten(),  # Input: 64@4×4, Output: 1024
            Linear(1024, 64),  # Input: 1024, Output: 64
            Linear(64, 10)  # Input: 64, Output: 10
        )

    def forward(self, inputs):
        outputs = self.model(inputs)
        return outputs


module = MyModule()
print(module)

MyModule(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [3]:
transformer = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

DATA_DIR = '../05-Transforms/data/'

test_data = torchvision.datasets.CIFAR10(
    root=DATA_DIR,
    transform=transformer,
    train=False,
    download=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_data,
    batch_size=1,
    shuffle=True,
    num_workers=4,
    drop_last=True
)

Files already downloaded and verified


In [10]:
loss = CrossEntropyLoss()
optim = torch.optim.SGD(module.parameters(), lr=0.01)

In [7]:
for i, data in enumerate(test_loader):
    imgs, targets = data
    outputs = module(imgs)
    res_loss = loss(outputs, targets)
    optim.zero_grad()
    res_loss.backward()
    optim.step()
    print(res_loss)

# 只经过一轮遍历学习的话，感觉loss也并没有很明显的下降，感觉用的是随机梯度下降？？？

tensor(2.3862, grad_fn=<NllLossBackward0>)
tensor(1.5154, grad_fn=<NllLossBackward0>)
tensor(2.8242, grad_fn=<NllLossBackward0>)
tensor(1.5649, grad_fn=<NllLossBackward0>)
tensor(2.2648, grad_fn=<NllLossBackward0>)
tensor(1.4820, grad_fn=<NllLossBackward0>)
tensor(1.1302, grad_fn=<NllLossBackward0>)
tensor(0.5791, grad_fn=<NllLossBackward0>)
tensor(0.6282, grad_fn=<NllLossBackward0>)
tensor(1.1096, grad_fn=<NllLossBackward0>)
tensor(3.0645, grad_fn=<NllLossBackward0>)
tensor(3.7498, grad_fn=<NllLossBackward0>)
tensor(2.7939, grad_fn=<NllLossBackward0>)
tensor(3.2183, grad_fn=<NllLossBackward0>)
tensor(3.7977, grad_fn=<NllLossBackward0>)
tensor(0.3727, grad_fn=<NllLossBackward0>)
tensor(0.1879, grad_fn=<NllLossBackward0>)
tensor(1.8064, grad_fn=<NllLossBackward0>)
tensor(2.1196, grad_fn=<NllLossBackward0>)
tensor(1.7065, grad_fn=<NllLossBackward0>)
tensor(3.2137, grad_fn=<NllLossBackward0>)
tensor(1.3681, grad_fn=<NllLossBackward0>)
tensor(1.7181, grad_fn=<NllLossBackward0>)
tensor(1.17

In [ ]:
loss = CrossEntropyLoss()
optim = torch.optim.SGD(module.parameters(), lr=0.001)

In [13]:
for epoch in range(20):
    epoch_loss = 0  # 对每一轮的loss进行累加，观察经过多轮学习后，loss是否有下降
    for i, data in enumerate(test_loader):
        imgs, targets = data
        outputs = module(imgs)
        res_loss = loss(outputs, targets)
        optim.zero_grad()
        res_loss.backward()
        optim.step()
        epoch_loss += res_loss

    print(epoch_loss)

tensor(15478.3701, grad_fn=<AddBackward0>)
tensor(15637.2529, grad_fn=<AddBackward0>)
tensor(16672.3066, grad_fn=<AddBackward0>)
tensor(17938.5332, grad_fn=<AddBackward0>)
tensor(19464.2578, grad_fn=<AddBackward0>)
tensor(20699.1191, grad_fn=<AddBackward0>)
tensor(22089.1680, grad_fn=<AddBackward0>)
tensor(23072.8750, grad_fn=<AddBackward0>)
tensor(23931.6953, grad_fn=<AddBackward0>)
tensor(24684.4609, grad_fn=<AddBackward0>)
tensor(25423.6172, grad_fn=<AddBackward0>)
tensor(26170.1484, grad_fn=<AddBackward0>)
tensor(26751.4668, grad_fn=<AddBackward0>)


KeyboardInterrupt: 